In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
index=pd.read_csv("../input/categories/index.csv")

In [3]:
index.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567385_00026,Complete,https://transcription.si.edu/transcribe/8489/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/8489/image/NMAAHC-004567385_00026.jpg,1,"NAME INDEX TO VOLUME 7 OF LETTERS SENT, JULY 1...",/8489/transcriptions/NMAAHC-004567385_00026.txt,\N,\N,8489,Index,NaN,NaN
1,NMAAHC-004567385_00027,Complete,https://transcription.si.edu/transcribe/8489/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/8489/image/NMAAHC-004567385_00027.jpg,1,"Boyd, J. F. (Col.) 19, 41\r\nBoyden, ...",/8489/transcriptions/NMAAHC-004567385_00027.txt,\N,\N,8489,Index,NaN,NaN
2,NMAAHC-004567385_00028,Complete,https://transcription.si.edu/transcribe/8489/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/8489/image/NMAAHC-004567385_00028.jpg,1,"Chur, J. F. (Capt.) 139\r\nCilley, Cli...",/8489/transcriptions/NMAAHC-004567385_00028.txt,\N,\N,8489,Index,NaN,NaN
3,NMAAHC-004567385_00029,Complete,https://transcription.si.edu/transcribe/8489/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/8489/image/NMAAHC-004567385_00029.jpg,1,"Elijah 176\r\nEliza 172\r\nEmery, Charles (Cap...",/8489/transcriptions/NMAAHC-004567385_00029.txt,\N,\N,8489,Index,NaN,NaN
4,NMAAHC-004567385_00030,Complete,https://transcription.si.edu/transcribe/8489/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/8489/image/NMAAHC-004567385_00030.jpg,1,"Hale, (Capt.) 51\r\nHambrich, P.R. (Capt.) 9\r...",/8489/transcriptions/NMAAHC-004567385_00030.txt,\N,\N,8489,Index,NaN,NaN


In [4]:
index.shape

(5422, 15)

In [5]:
index_text=index["transcription_text"]

In [6]:
print(index_text[0])

NAME INDEX TO VOLUME 7 OF LETTERS SENT, JULY 1865-JULY 1867. PAGES 1-179

Abbott, Lyman (Rev.)        26
Adams, John Q.              37
Allen, Moses H.             5
Almy, George C. (Capt.)     128
Anderson, James H. (Lt.)    88, 91, 121
Archy, Mose                 173
Armsby, Lawrence (Chaplain) 4
Armstrong, S. C.            112
Bardham, H. A. (Atty.)      178
Barnes, Haywood             93
Barnes, William             39
Barnitt, John C. (Capt.)    4, 8, 22, 36, 49, 50, 69, 74, 97
Barringer, Victor C.        172
Beadle, W. H. H. (Bvt. Lt. Col.) 90, 95
Beal (Capt.)                     2
Beal, James H.                   100
Beath, Robert B. (Capt.)         14
Beecher, Fred H. (Lt.)           92, 121
Bell, S. A. (A. A. Surg.)        153
Bellamy, John D.                 56
Binning (Lt.)                    164
Birdsall (Lt.)                   20, 21
Blunt, Hargate and Blunt         42
Bond, Henry A.                   175
Bowen, John J. (Capt.)           1


In [7]:
print(index_text[10])

[[on page underlying page with index entries]]

[[underlined]] No. Ca. [[/underlined]]

Name [[underlined]] index to No 7. [[/underlined]]

(part of 1866 and to July 13/67)

[[in the right margin]] (8)

[[/underlying page]]

Letters Sent

Names | Index of Letters | Names | Index of Letters
----- | ------- | ------- | ------ 
"A" | [[no head]] | "B" | [[no head]]
Armsby. Lawrence Chaplain | P.4. No 8. | Bullock Jas. A. | P. 2-4
Adams Jno. L. | P.37.88.- | Barnett John C Capt & Supt. | P. 4. No 7 - P. 8 No.16-36.86.-49.116.-50.118-69.158.-97.223.-
Anderson Lt. Jas. H. | 88.198-91.207.-114.261.-121.277.- | Brown R.B. | P. [[3?]]: No. 9
Almy Geo. C. | 102.232.-108.247.-128.291.- | Boyd. J.[[F.?]] Col. & Chf. Q.M.
Armstrong S.C.  | 112.256.- | Blunt, Heargate + Blunt | P. 42. No. 102-
[[strikethrough]] Aarons, W. [[/strikethrough]] |  [[blank]] | Buxton Ralph P. | 87.197.-
[[blank]] | [[blank]] | Bryan Henry R. | 89.202-
[[blank]] | [[blank]] | Beadle Lt. Col. W.H.H. | 90-204.-98.212.-95.21

In [8]:
print(index_text[100])

Greenlee. John a.  72.
Goodding. Isaac    85.
Green. Rosa   85.
Green. Joseph 85.


In [9]:
print(index_text[150])

[[start page]]
1865.
[[three column table]]

London Manger | [[blank]] | 14.
London Sarah E | [[blank]] | 16.
Lippett John E | [[blank]] | 18.
Lippett W.H | [[blank]] | 18. 23. 33.
Laslo J.G | [[blank]] | 19.
Lehman R.F | [[blank]] | 20. 44.
Lecraft Mary | [[blank]] | 21.
Lazarus Julia | [[blank]] | 22.
Lippett W. | [[blank]] | 23. 33.
Lecraft Benjamin | [[blank]] | 36. 59.
Lewis K.H | [[blank]] | 30.
Lewis W.F | [[blank]] | 30.
Levy T.J. | [[blank]] | 31.
Lane H.B | [[blank]] | 37.
Leach J A | [[blank]] | 40. 45.
Leavy J.P | [[blank]] | 50.
Lord Fredk | [[blank]] | 52.
Littlejohn W.H | [[blank]] | 53.
Leigh J.W | [[blank]] | 66.
Low T.B | [[blank]] | 66.
[[end page]]


In [10]:
print(index_text[5000])

[[Right Hand Page]]


[[2 Columned Table]]
--- | --- 
"Jenny" | 90. 
Johnson  Richard | 10
Jones  Andrew  | 12
Johnston  Thos. P. A.N.U. | 16, 20, 75
Jordan  Alice |  37, 53
Jordan  Vina |  51, 54, 55  
Johnston  Tho.P. A.N.U. | 65, 74 95, 169, 199, 200, 204, 205, 281
Johnston  Hugh | 107, 146 
"Jane" | 129
"Jennings"  Mr | 184, 185
Johnston  David | 219
Johnson  Leah | 228
Jeremia & Parthe[[?]] | 253


In [11]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\[]).*?([\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
start=time.time()
for i in tqdm(range(len(index_text))):
    index_text[i] = process(index_text[i])



100%|██████████| 5422/5422 [00:31<00:00, 170.63it/s]


In [12]:
for i in tqdm(range(len(index_text))):
    index_text[i] = " ".join(index_text[i])
end=time.time()
print(end-start)

100%|██████████| 5422/5422 [00:02<00:00, 2687.89it/s]

33.869304180145264


In [13]:
index_text

0       NAME INDEX TO VOLUME 7 OF LETTERS SENT , JULY ...
1       Boyd , J. F. Col. 19 , 41 Boyden , James 116 B...
2       Chur , J. F. Capt . 139 Cilley , Clinton A. Ma...
3       Elijah 176 Eliza 172 Emery , Charles Capt . 3 ...
4       Hale , Capt . 51 Hambrich , P.R . Capt . 9 Han...
                              ...                        
5417                    Names Locations of Teachers Ala .
5418    THE NATIONAL ARCHIVES OF THE UNITED STATES Nam...
5419    Continued . Hattie L. Newcomb Demopolis Maggie...
5420    Continued . Hattie L. Newcomb Demopolis Maggie...
5421    Continued W. W. Oslim Oak Bowery James Nichols...
Name: transcription_text, Length: 5422, dtype: object

In [14]:
print(index_text[0])

NAME INDEX TO VOLUME 7 OF LETTERS SENT , JULY 1865JULY 1867 . PAGES 1179 Abbott , Lyman Rev . 26 Adams , John Q . 37 Allen , Moses H. 5 Almy , George C. Capt . 128 Anderson , James H. Lt. 88 , 91 , 121 Archy , Mose 173 Armsby , Lawrence Chaplain 4 Armstrong , S. C. 112 Bardham , H. A. Atty . 178 Barnes , Haywood 93 Barnes , William 39 Barnitt , John C. Capt . 4 , 8 , 22 , 36 , 49 , 50 , 69 , 74 , 97 Barringer , Victor C. 172 Beadle , W. H. H. Bvt . Lt. Col. 90 , 95 Beal Capt . 2 Beal , James H. 100 Beath , Robert B. Capt . 14 Beecher , Fred H. Lt. 92 , 121 Bell , S. A . A . A. Surg . 153 Bellamy , John D. 56 Binning Lt. 164 Birdsall Lt. 20 , 21 Blunt , Hargate Blunt 42 Bond , Henry A . 175 Bowen , John J. Capt . 1
